In [1]:
import import_parent

In [2]:
import tree_sim
tree_sim.EXPIRATION_TIME = 1e2
tree_sim.MEMORIES_PER_END_NODE=30

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import plot_sim_results

In [5]:
import ipywidgets

In [6]:
from multiprocessing import Pool, cpu_count
import os
from shutil import rmtree
import pickle
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm as tqdm
%matplotlib inline

In [7]:
dir_str = "jupyter_example_data"
pool = Pool()

In [15]:
def run_sim_with_size(n, k, b, samples):
    rmtree(dir_str)
    os.makedirs(dir_str)
    l=cpu_count()
    if samples == 11:
        samples = True
        print("WARNING: Sampling until convergence.  Expect long runtimes (on the order of minutes)")
    elif samples == 0:
        samples = False
        
    params = [{"p" : 0.1**(1+2*i/l), "k" : 3, "n" : n, "t" : "2e3", "b" : b, "resample" : samples} for i in range(l)]
    for result in tqdm(pool.imap_unordered(tree_sim.launch_sim_from_dict, params), total=l):
        os.makedirs(os.path.join(dir_str, f"n_{result['init_data'][2]}"), exist_ok=True) # The data is organized into subfolders based on the value of n.
        with open(os.path.join(dir_str, f"n_{result['init_data'][2]}/{result['init_data']}.data"), "wb") as f: # Each individual simulation has a data file, with a name based on its parameters.
            pickle.dump(result, f)

    plotdict = {
            "x" : "p",
            "y" : "rate",
            "indir" : dir_str,
            "legend" : True,
            }
    plot_sim_results.make_plot_from_dict(plotdict)
    plt.show()

print("Explanation:")
print("n: The number of layers of the tree.")
print("k: The branching factor the tree.")
print("b: The word size of requests.")
print("samples: The number of samples taken for averaging.  At samples=11 it will instead sample until convergence.")
print("N = k^(n-1) is the number of client nodes served by the tree.")
interactive_sim = ipywidgets.interactive(run_sim_with_size, n=ipywidgets.IntSlider(3, 2, 4, 1), k=ipywidgets.IntSlider(3,2,4,1), b=ipywidgets.IntSlider(1,1,8,1), samples=ipywidgets.IntSlider(5,1,11,1))
interactive_sim

Explanation:
n: The number of layers of the tree.
k: The branching factor the tree.
b: The word size of requests.
samples: The number of samples taken for averaging.  At samples=11 it will instead sample until convergence.
N = k^(n-1) is the number of client nodes served by the tree.


interactive(children=(IntSlider(value=3, description='n', max=4, min=2), IntSlider(value=3, description='k', m…